## previo

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
!pip install rasterio
import rasterio
!pip install opencv-python
import cv2
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

# Configuración de rutas
DATA_DIR = "/content/drive/MyDrive/Imagen_unet/1a"  #para 1 año
#DATA_DIR = "/content/drive/MyDrive/Imagen_unet/part1"


PRECIPITATION_DIR = os.path.join(DATA_DIR, "Precipitacion")
TEMPERATURE_DIR = os.path.join(DATA_DIR, "Temperatura")
INCIDENCE_DIR = os.path.join(DATA_DIR, "NBI.png")
#NBI_MISE_DIR = os.path.join(DATA_DIR, "nbi_mise")
#NBI_SERV_DIR = os.path.join(DATA_DIR, "nbi_serv")
NBI_MISE= os.path.join(DATA_DIR, "nbi_mise_2005.png")
NBI_SERV = os.path.join(DATA_DIR, "nbi_serv_2005.png")
DEF_AGUA = os.path.join(DATA_DIR, "def_agua.png")
DEF_ALC = os.path.join(DATA_DIR, "def_alc.png")
DEF_BAS = os.path.join(DATA_DIR, "def_bas.png")
POBREZA = os.path.join(DATA_DIR, "pobreza_agua.png")

## dataset

Se ordena la data para ingresarla al modelo

In [ ]:
class DengueDataset(Dataset):
    def __init__(self, data_dir=DATA_DIR, mode='train', use_precipitation=True,
                 use_temperature=True, use_nbi_mise=True, use_nbi_serv=True,
                 use_def_agua=True, use_def_alc=True, use_def_bas=True,
                 use_pobreza = True, target_size=(512, 512), train_split=0.7, val_split=0.15):
        self.data_dir = data_dir
        self.mode = mode
        self.use_precipitation = use_precipitation
        self.use_temperature = use_temperature
        self.use_nbi_mise = use_nbi_mise
        self.use_nbi_serv = use_nbi_serv
        self.use_def_agua = use_def_agua
        self.use_def_alc = use_def_alc
        self.use_def_bas = use_def_bas
        self.use_pobreza = use_pobreza
        self.target_size = target_size
        self.train_split = train_split
        self.val_split = val_split

        # Numero de canales
        self.n_channels = 0
        if self.use_precipitation:
            self.n_channels += 1
        if self.use_temperature:
            self.n_channels += 1
        if self.use_nbi_mise:
            self.n_channels += 1
        if self.use_nbi_serv:
            self.n_channels += 1
        if self.use_def_agua:
            self.n_channels += 1
        if self.use_def_alc:
            self.n_channels += 1
        if self.use_def_bas:
            self.n_channels += 1
        if self.use_pobreza:
            self.n_channels += 1

        # Obtener y dividir los nombres base
        self.basename_list = self._split_data()

        # Cargar imagenes
        if self.use_nbi_mise:
            self.nbi_mise = self._load_nbi_mise()
        if self.use_nbi_serv:
            self.nbi_serv = self._load_nbi_serv()
        if self.use_def_agua:
            self.def_agua = self._load_def_agua()
        if self.use_def_alc:
             self.def_alc = self._load_def_alc()
        if self.use_def_bas:
            self.def_bas = self._load_def_bas()
        if self.use_pobreza:
            self.pobreza = self._load_pobreza()

    def _get_basename_list(self):
        """
        Obtiene los nombres base (año.semana) presentes en todas las carpetas necesarias.
        """
        precipitation_files = set(os.path.splitext(os.path.basename(f))[0] for f in
                                   os.listdir(os.path.join(self.data_dir, "Precipitacion")))
        temperature_files = set(os.path.splitext(os.path.basename(f))[0] for f in
                                 os.listdir(os.path.join(self.data_dir, "Temperatura")))
        incidence_files = set(os.path.splitext(os.path.basename(f))[0] for f in
                              os.listdir(os.path.join(self.data_dir, "Incidencia")))

        # Ordenar por fecha
        base_names = sorted(list(precipitation_files & temperature_files & incidence_files))
        #print(base_names)
        return base_names

    def _split_data(self):
        """
        Divide los datos en entrenamiento, validación y prueba según los porcentajes deseados.
        """
        base_names = self._get_basename_list()

        # Dividir datos en conjunto de entrenamiento, validación y prueba
        train_size = int(len(base_names) * self.train_split)
        val_size = int(len(base_names) * self.val_split)
        train = base_names[:train_size]
        val = base_names[train_size:train_size + val_size]
        test = base_names[train_size + val_size:]
        print("Prueba:", test)

        if self.mode == 'train':
            return train
        elif self.mode == 'val':
            return val
        elif self.mode == 'test':
            return test
        else:
            return base_names

    def _load_image(self, folder, basename, value_range):
        """
        Carga y normalizar imagenes
        """
        filename = os.path.join(self.data_dir, folder, f"{basename}.tif")
        img = Image.open(filename).convert("L").resize(self.target_size, Image.BILINEAR)
        data = np.array(img).astype(np.float32) / value_range[1]
        return data

    def _load_precipitation(self, basename):
        """
        Carga las imágenes de precipitación normalizadas.
        """
        return self._load_image("Precipitacion", basename, (0.0, 112))

    def _load_temperature(self, basename):
        """
        Carga las imágenes de temperatura normalizadas.
        """
        return self._load_image("Temperatura", basename, (13000, 16500))

    def _load_incidence(self, basename):
        """
        Carga las imágenes de incidencia normalizadas.
        """
        filename = os.path.join(self.data_dir, "Incidencia", f"{basename}.png")
        if not os.path.exists(filename):
            raise FileNotFoundError(f"El archivo {filename} no existe.")
        try:
            img = Image.open(filename).convert("L").resize(self.target_size, Image.BILINEAR)
            data = np.array(img).astype(np.float32)
            data_normalized = (data - data.min()) / (data.max() - data.min())
            thresholds = [0.2, 0.4, 0.5, 0.8]
            categorized_labels = np.digitize(data_normalized, thresholds)
            return categorized_labels

        except Exception as e:
            print(f"Error al procesar {filename}: {e}")
            return None

    # Se cargan las imágenes normalizadas para todas las variables sociales
    def _load_nbi_mise(self):
        filename = os.path.join(self.data_dir, "nbi_mise_2005.png")
        img = Image.open(filename).convert("L").resize(self.target_size, Image.BILINEAR)
        data = np.array(img).astype(np.float32)
        data_normalized = (data - data.min()) / (data.max() - data.min())
        thresholds = [0.25, 0.5, 0.75]
        categorized_labels = np.digitize(data_normalized, thresholds)
        return categorized_labels

    def _load_nbi_serv(self):
        filename = os.path.join(self.data_dir, "nbi_serv_2005.png")
        img = Image.open(filename).convert("L").resize(self.target_size, Image.BILINEAR)
        data = np.array(img).astype(np.float32)
        data_normalized = (data - data.min()) / (data.max() - data.min())
        thresholds = [0.25, 0.5, 0.75]
        categorized_labels = np.digitize(data_normalized, thresholds)
        return categorized_labels

    def _load_def_agua(self):
        filename = os.path.join(self.data_dir, "def_agua.png")
        img = Image.open(filename).convert("L").resize(self.target_size, Image.BILINEAR)
        data = np.array(img).astype(np.float32)
        data_normalized = (data - data.min()) / (data.max() - data.min())
        thresholds = [0.25, 0.5, 0.75]
        categorized_labels = np.digitize(data_normalized, thresholds)
        return categorized_labels

    def _load_def_alc(self):
        filename = os.path.join(self.data_dir, "def_alc.png")
        img = Image.open(filename).convert("L").resize(self.target_size, Image.BILINEAR)
        data = np.array(img).astype(np.float32)
        data_normalized = (data - data.min()) / (data.max() - data.min())
        thresholds = [0.25, 0.5, 0.75]
        categorized_labels = np.digitize(data_normalized, thresholds)
        return categorized_labels

    def _load_def_bas(self):
        filename = os.path.join(self.data_dir, "def_bas.png")
        img = Image.open(filename).convert("L").resize(self.target_size, Image.BILINEAR)
        data = np.array(img).astype(np.float32)
        data_normalized = (data - data.min()) / (data.max() - data.min())
        thresholds = [0.25, 0.5, 0.75]
        categorized_labels = np.digitize(data_normalized, thresholds)
        return categorized_labels

    def _load_pobreza(self):
        filename = os.path.join(self.data_dir, "pobreza_agua.png")
        img = Image.open(filename).convert("L").resize(self.target_size, Image.BILINEAR)
        data = np.array(img).astype(np.float32)
        data_normalized = (data - data.min()) / (data.max() - data.min())
        thresholds = [0.25, 0.5, 0.75]
        categorized_labels = np.digitize(data_normalized, thresholds)
        return categorized_labels

    def __getitem__(self, idx):
        """
        Carga datos específicos (features y label).
        """
        basename = self.basename_list[idx]
        data = []

        if self.use_precipitation:
            data.append(self._load_precipitation(basename))

        if self.use_temperature:
            data.append(self._load_temperature(basename))

        if self.use_nbi_mise:
            data.append(self.nbi_mise)

        if self.use_nbi_serv:
            data.append(self.nbi_serv)

        if self.use_def_agua:
            data.append(self.def_agua)

        if self.use_def_alc:
            data.append(self.def_alc)

        if self.use_def_bas:
            data.append(self.def_bas)

        if self.use_pobreza:
            data.append(self.pobreza)

        # Apilar todos los canales
        data = np.stack(data).astype(np.float32)
        data = torch.tensor(data)


        label = self._load_incidence(basename)
        label = torch.tensor(label, dtype=torch.long)

        return data, label

    def __len__(self):
        return len(self.basename_list)


## model

In [ ]:
def _make_layers(in_channels, config, batch_norm=True):
    _layers = []
    _in_ch = in_channels
    for _cfg in config:
        if _cfg == 'M':
            _layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            if batch_norm:
                _layers += [nn.Conv2d(_in_ch, _cfg, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(_cfg),
                            nn.ReLU(inplace=True)]
            else:
                _layers += [nn.Conv2d(_in_ch, _cfg, kernel_size=3, stride=1, padding=1),
                            nn.ReLU(inplace=True)]
            _in_ch = _cfg
    return nn.Sequential(*_layers)

class UNet(nn.Module):
    def __init__(self, in_channels=8, n_class=5):
    #def __init__(self, in_channels=3, n_class=4):
        super().__init__()

        self.in_channels = in_channels
        self.n_class = n_class

        # encoder
        self.enc1 = _make_layers(self.in_channels, [64, 64])
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.enc2 = _make_layers(64, [128, 128])
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.enc3 = _make_layers(128, [256, 256])
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.enc4 = _make_layers(256, [512, 512])
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        # center
        self.center = _make_layers(512, [1024, 1024])

        # decoder
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = _make_layers(1024, [512, 512])
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = _make_layers(512, [256, 256])
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = _make_layers(256, [128, 128])
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = _make_layers(128, [64, 64])

        # output
        self.score = nn.Conv2d(64, self.n_class, kernel_size=1, stride=1, padding=0)

        self._initialize_weight()

    def forward(self, x):

        # encoder
        e1 = self.enc1(x)
        h = self.pool1(e1)
        e2 = self.enc2(h)
        h = self.pool2(e2)
        e3 = self.enc3(h)
        h = self.pool3(e3)
        e4 = self.enc4(h)
        h = self.pool4(e4)

        # center
        h = self.center(h)

        # decoder
        h = self.up4(h)
        h = self.dec4(torch.cat([e4, h], 1))
        h = self.up3(h)
        h = self.dec3(torch.cat([e3, h], 1))
        h = self.up2(h)
        h = self.dec2(torch.cat([e2, h], 1))
        h = self.up1(h)
        h = self.dec1(torch.cat([e1, h], 1))

        # output
        h = self.score(h)

        return h

    def _initialize_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.ConvTranspose2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)


#### .

In [ ]:
class RunningScore(object):

    def __init__(self, n_classes):
        self.n_classes = n_classes
        self.confusion_matrix = np.zeros((n_classes, n_classes))

    def _fast_hist(self, label_true, label_pred, n_class):
        mask = (label_true >= 0) & (label_true < n_class)
        hist = np.bincount(
            n_class * label_true[mask].astype(int) + label_pred[mask], minlength=n_class ** 2
        ).reshape(n_class, n_class)
        return hist

    def update(self, label_trues, label_preds):
        for lt, lp in zip(label_trues, label_preds):
            self.confusion_matrix += self._fast_hist(lt.flatten(), lp.flatten(), self.n_classes)

    def get_scores(self):
        """Returns accuracy score evaluation result.
            - overall accuracy
            - mean accuracy
            - mean IU
            - fwavacc
        """
        hist = self.confusion_matrix
        acc = np.diag(hist).sum() / hist.sum()
        acc_cls = np.diag(hist) / hist.sum(axis=1)
        acc_cls = np.nanmean(acc_cls)
        iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
        mean_iu = np.nanmean(iu)
        freq = hist.sum(axis=1) / hist.sum()
        fwavacc = (freq[freq > 0] * iu[freq > 0]).sum()
        cls_iu = dict(zip(range(self.n_classes), iu))

        # dice
        if self.n_classes == 5:
            dice = self.confusion_matrix[1, 1] / (self.confusion_matrix[1, 1] + 0.5 * (self.confusion_matrix[0, 1] + self.confusion_matrix[1, 0]))
        else:
            dice = 0.0

        return (
            {
                "OverallAcc": acc,
                "MeanAcc": acc_cls,
                "FreqWAcc": fwavacc,
                "MeanIoU": mean_iu,
                "Dice": dice,
            },
            cls_iu,
        )

    def reset(self):
        self.confusion_matrix = np.zeros((self.n_classes, self.n_classes))


In [ ]:
def evaluation(model, train_loader, use_cuda):
    eval_metrics = RunningScore(n_classes=5)
    model.eval()
    with torch.no_grad():
        for data, target in train_loader:
            if use_cuda:
                data = data.cuda()

            output = model(data)

            if use_cuda:
                pred = output.data.max(1)[1].cpu().numpy()
            else:
                pred = output.data.max(1)[1].numpy()

            eval_metrics.update(target.numpy(), pred)

    score, class_iou = eval_metrics.get_scores()

    print("Overall score:\n", score)
    print("Class IoU:\n", class_iou, "\n")

    return score, class_iou


## train

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dataset = DengueDataset(data_dir=DATA_DIR, mode='train', use_precipitation=True,
                 use_temperature=True, use_nbi_mise=True, use_nbi_serv=True,
                 use_def_agua=True, use_def_alc=True, use_def_bas=True,
                 use_pobreza = True)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

val_dataset = DengueDataset(data_dir=DATA_DIR, mode='val', use_precipitation=True,
                 use_temperature=True, use_nbi_mise=True, use_nbi_serv=True,
                 use_def_agua=True, use_def_alc=True, use_def_bas=True,
                 use_pobreza = True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

test_dataset = DengueDataset(data_dir=DATA_DIR, mode='test', use_precipitation=True,
                 use_temperature=True, use_nbi_mise=True, use_nbi_serv=True,
                 use_def_agua=True, use_def_alc=True, use_def_bas=True,
                 use_pobreza = True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)
model = UNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Prueba: ['2007.w44', '2007.w45', '2007.w46', '2007.w47', '2007.w48', '2007.w49', '2007.w50', '2007.w51', '2007.w52']
Prueba: ['2007.w44', '2007.w45', '2007.w46', '2007.w47', '2007.w48', '2007.w49', '2007.w50', '2007.w51', '2007.w52']
Prueba: ['2007.w44', '2007.w45', '2007.w46', '2007.w47', '2007.w48', '2007.w49', '2007.w50', '2007.w51', '2007.w52']


In [ ]:
# Calcular pesos para las clases
def calculate_class_counts(data_loader):
    class_counts = torch.zeros(5)
    for _, labels in data_loader:
        unique, counts = torch.unique(labels, return_counts=True)
        for u, c in zip(unique, counts):
            class_counts[u] += c
    return class_counts

# Calcular pesos
#class_counts = calculate_class_counts(train_loader)
#print(class_counts)
#class_weights = 1.0 / class_counts
#class_weights = class_weights / class_weights.sum()
#class_weights = class_weights.to(device)
#print(class_weights)

Pesos obtenidos para el periodo de 2007-2018:



```
print(class_counts)
```



tensor([77770512.,   344566.,   206455.,  1948665., 34024596.])


```
print(class_weights)
```


tensor([0.0015, 0.3496, 0.5835, 0.0618, 0.0035], device='cuda:0')

In [ ]:
# Asignar pesos a las clases de forma manual
class_weights = torch.tensor([0.0015, 0.3496, 0.5835, 0.0618, 0.0035], device='cuda:0')
class_weights

tensor([0.0015, 0.3496, 0.5835, 0.0618, 0.0035], device='cuda:0')

In [ ]:
writer = SummaryWriter()

def train_model(model, train_loader, val_loader, device, epochs=10, lr=0.0001): #e = 40 l
    use_cuda = torch.cuda.is_available()
    criterion = nn.CrossEntropyLoss()
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    best_score = 0.0

    model.train()
    for epoch in range(1, epochs + 1):
        print(f"Epoch: {epoch}")
        model.train()
        train_loss = 0  # Reinicializar train_loss en cada época

        for data, target in train_loader:
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        print(f"Train Loss: {avg_train_loss}")

        # Evaluación cada 5 épocas
        if epoch % 5 == 0:
            print("\nEvaluation ...")
            val_score, _ = evaluation(model, val_loader, use_cuda)
            writer.add_scalar('val_accuracy/overall_acc', val_score["OverallAcc"], epoch)
            writer.add_scalar('val_accuracy/mean_iou', val_score["MeanIoU"], epoch)
            writer.add_scalar('val_accuracy/dice', val_score["Dice"], epoch)

            if best_score < val_score["Dice"]:
                best_score = val_score["Dice"]
                torch.save(model.state_dict(), os.path.join("snapshot-best.pt"))

        if epoch % 10 == 0:
            torch.save(model.state_dict(), os.path.join(f"snapshot-{epoch:04d}.pt"))

    return model.state_dict()


In [ ]:
train_model(model, train_loader, val_loader, device)

Epoch: 1
Train Loss: 2.6227413747045727
Epoch: 2
Train Loss: 0.35174993177254993
Epoch: 3
Train Loss: 0.22855880690945518
Epoch: 4
Train Loss: 0.1814316279358334
Epoch: 5
Train Loss: 0.16192520161469778

Evaluation ...
Overall score:
 {'OverallAcc': np.float64(0.5955614362444196), 'MeanAcc': np.float64(0.43478642330338635), 'FreqWAcc': np.float64(0.5528656207168386), 'MeanIoU': np.float64(0.3269022858740993), 'Dice': np.float64(0.4650524389385514)}
Class IoU:
 {0: np.float64(0.47006087161987303), 1: np.float64(0.24576870140981288), 2: np.float64(0.8479916388131493), 3: np.float64(0.07069021752766139), 4: np.float64(0.0)} 

Epoch: 6
Train Loss: 0.147827610373497
Epoch: 7
Train Loss: 0.13525569604502785
Epoch: 8
Train Loss: 0.12777266237470838
Epoch: 9
Train Loss: 0.12443174670139949
Epoch: 10
Train Loss: 0.1211539399292734

Evaluation ...
Overall score:
 {'OverallAcc': np.float64(0.9099399021693638), 'MeanAcc': np.float64(0.569286782709106), 'FreqWAcc': np.float64(0.8706396096380208), '

OrderedDict([('enc1.0.weight',
              tensor([[[[ 0.0618, -0.0455, -0.0855],
                        [-0.0116,  0.0713, -0.1726],
                        [-0.0770,  0.0582,  0.0733]],
              
                       [[ 0.0845,  0.0274,  0.0122],
                        [-0.1535,  0.0565, -0.0858],
                        [-0.0626,  0.0037,  0.0313]],
              
                       [[-0.0302, -0.0443,  0.0594],
                        [ 0.0150,  0.0635,  0.0419],
                        [-0.0862,  0.0497,  0.0341]],
              
                       ...,
              
                       [[-0.0714, -0.0373, -0.0082],
                        [ 0.0291, -0.1156, -0.1969],
                        [-0.0775, -0.0837, -0.0622]],
              
                       [[-0.0104,  0.0320,  0.0031],
                        [-0.1499, -0.0109,  0.0420],
                        [ 0.0629,  0.0891,  0.0351]],
              
                       [[ 0.0416,  0.0287, -0.0619]

## loss y metrics

In [ ]:
evaluation(model, test_loader, device)

Overall score:
 {'OverallAcc': np.float64(0.9181594848632812), 'MeanAcc': np.float64(0.561869135455186), 'FreqWAcc': np.float64(0.8849235864766831), 'MeanIoU': np.float64(0.4431178673894438), 'Dice': np.float64(0.5169126691266913)}
Class IoU:
 {0: np.float64(0.9295434031988279), 1: np.float64(0.2728453173186171), 2: np.float64(0.8872594888393127), 3: np.float64(0.12594112759046155), 4: np.float64(0.0)} 



({'OverallAcc': np.float64(0.9181594848632812),
  'MeanAcc': np.float64(0.561869135455186),
  'FreqWAcc': np.float64(0.8849235864766831),
  'MeanIoU': np.float64(0.4431178673894438),
  'Dice': np.float64(0.5169126691266913)},
 {0: np.float64(0.9295434031988279),
  1: np.float64(0.2728453173186171),
  2: np.float64(0.8872594888393127),
  3: np.float64(0.12594112759046155),
  4: np.float64(0.0)})

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

def evaluate_model(data_loader, model, device):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            predictions = model(inputs)
            predictions = torch.argmax(predictions, dim=1).cpu().numpy()
            all_labels.extend(labels.cpu().numpy().flatten())
            all_preds.extend(predictions.flatten())
    print(confusion_matrix(all_labels, all_preds))
    print(classification_report(all_labels, all_preds, target_names=["Clase 0", "Clase 1", "Clase 2", "Clase 3", "Clase 4"]))

evaluate_model(test_loader, model, device)


[[1487117   18852    7115    4395   82357]
 [      0   10086    3408       7       3]
 [      0    1546  659004    2038       0]
 [      0    2774   59538   10003       2]
 [      0     290   10092     669       0]]
              precision    recall  f1-score   support

     Clase 0       1.00      0.93      0.96   1599836
     Clase 1       0.30      0.75      0.43     13504
     Clase 2       0.89      0.99      0.94    662588
     Clase 3       0.58      0.14      0.22     72317
     Clase 4       0.00      0.00      0.00     11051

    accuracy                           0.92   2359296
   macro avg       0.56      0.56      0.51   2359296
weighted avg       0.95      0.92      0.93   2359296



In [ ]:
def calculate_iou_and_dice(test_loader, model, device, num_classes=5):
    """
    Calcula la Intersección sobre Unión (IoU) y el Dice Coefficient por clase.
    Args:
        test_loader: DataLoader para el conjunto de prueba.
        model: Modelo entrenado.
        device: CPU o GPU.
        num_classes: Número de clases.
    """
    model.eval()
    iou_per_class = np.zeros(num_classes)
    dice_per_class = np.zeros(num_classes)
    total_pixels_per_class = np.zeros(num_classes)

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            predictions = model(inputs)
            predicted_labels = torch.argmax(predictions, dim=1)

            for cls in range(num_classes):
                intersection = ((predicted_labels == cls) & (labels == cls)).sum().item()
                union = ((predicted_labels == cls) | (labels == cls)).sum().item()
                dice_denominator = (2 * intersection + ((predicted_labels == cls).sum().item() + (labels == cls).sum().item()))

                if union > 0:
                    iou_per_class[cls] += intersection / union
                if dice_denominator > 0:
                    dice_per_class[cls] += 2 * intersection / dice_denominator
                total_pixels_per_class[cls] += 1

    # Promediar por clase
    iou_per_class /= total_pixels_per_class
    dice_per_class /= total_pixels_per_class
    iou_global = np.mean(iou_per_class)
    print("IoU Global:", iou_global)
    print("IoU por Clase:", iou_per_class)
    print("Dice Global:", np.mean(dice_per_class))
    print("Dice Coefficient por Clase:", dice_per_class)

In [ ]:
calculate_iou_and_dice(test_loader, model, device, num_classes=5)

IoU Global: 0.45320073209862183
IoU por Clase: [0.92961845 0.27952916 0.9000857  0.15677034 0.        ]
Dice Global: 0.2975447123758705
Dice Coefficient por Clase: [0.49071203 0.30389377 0.48640809 0.20670968 0.        ]


## visualizacion

In [ ]:
import os
import matplotlib.pyplot as plt
import torch
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.plot import show
from rasterio.transform import from_bounds

# Cargar el shapefile de municipios
municipios = gpd.read_file("/content/drive/Shareddrives/JAVIERA_DENGUE/CODE/Shape")
municipios = municipios.to_crs("EPSG:4326")

# Leer el raster de predicciones
with rasterio.open("/content/drive/MyDrive/Predicciones/prediccion_test_0_0.tif") as src:
    pred = src.read(1)

# Establecer los límites esperados en coordenadas geográficas
raster_minx, raster_miny, raster_maxx, raster_maxy = -82, -4.5, -65, 12.5

# Crear una transformación espacial desde los límites
transform = from_bounds(raster_minx, raster_miny, raster_maxx, raster_maxy, pred.shape[1], pred.shape[0])

# Obtener límites geográficos ajustados
minx, miny = transform * (0, pred.shape[0])
maxx, maxy = transform * (pred.shape[1], 0)

In [ ]:
def visualize_test_predictions_with_borders(
    test_loader, model, municipios, transform, device, output_dir, dataset
):
    """
    Visualiza predicciones del modelo en el conjunto de prueba con bordes municipales y las compara con las etiquetas reales.
    """
    model.eval()  # Poner el modelo en modo de evaluación

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            predictions = model(inputs)
            predicted_labels = torch.argmax(predictions, dim=1).cpu().numpy()
            real_labels = labels.cpu().numpy()

            # Obtener los nombres base para el lote actual
            batch_basenames = dataset.basename_list[batch_idx * len(inputs):(batch_idx + 1) * len(inputs)]

            for i, basename in enumerate(batch_basenames):  # Iterar usando los nombres base
                # Crear una figura para visualizar las predicciones
                fig, axes = plt.subplots(1, 2, figsize=(14, 6))

                # Etiqueta real con bordes municipales
                show(real_labels[i], transform=transform, ax=axes[0], cmap="jet", title=f"Etiqueta Real ({basename})")
                municipios.boundary.plot(ax=axes[0], edgecolor="white", linewidth=0.4) ##0.4
                axes[0].set_xlim(minx, maxx)
                axes[0].set_ylim(miny, maxy)
                axes[0].axis("off")

                # Predicción del modelo con bordes municipales
                show(predicted_labels[i], transform=transform, ax=axes[1], cmap="jet", title=f"Predicción del Modelo ({basename})")
                municipios.boundary.plot(ax=axes[1], edgecolor="white", linewidth=0.4)
                axes[1].set_xlim(minx, maxx)
                axes[1].set_ylim(miny, maxy)
                axes[1].axis("off")

                # Guardar la figura usando el nombre base
                output_filename = os.path.join(output_dir, f"visualizacion_test_{basename}.png")
                plt.savefig(output_filename, dpi=300, bbox_inches="tight")
                plt.close(fig)

                print(f"Imagen guardada: {output_filename}")


In [ ]:
# Directorio de salida
output_dir = "/content/drive/MyDrive/Imagen_unet/Visualizaciones/prueba/año1"

# Llamada a la función
visualize_test_predictions_with_borders(
    test_loader=test_loader,
    model=model,
    municipios=municipios,
    transform=transform,
    device=device,
    output_dir=output_dir,
    dataset=test_dataset
)

Imagen guardada: /content/drive/MyDrive/Imagen_unet/Visualizaciones/prueba/año1/visualizacion_test_2007.w44.png
Imagen guardada: /content/drive/MyDrive/Imagen_unet/Visualizaciones/prueba/año1/visualizacion_test_2007.w45.png
Imagen guardada: /content/drive/MyDrive/Imagen_unet/Visualizaciones/prueba/año1/visualizacion_test_2007.w46.png
Imagen guardada: /content/drive/MyDrive/Imagen_unet/Visualizaciones/prueba/año1/visualizacion_test_2007.w47.png
Imagen guardada: /content/drive/MyDrive/Imagen_unet/Visualizaciones/prueba/año1/visualizacion_test_2007.w48.png
Imagen guardada: /content/drive/MyDrive/Imagen_unet/Visualizaciones/prueba/año1/visualizacion_test_2007.w49.png
Imagen guardada: /content/drive/MyDrive/Imagen_unet/Visualizaciones/prueba/año1/visualizacion_test_2007.w50.png
Imagen guardada: /content/drive/MyDrive/Imagen_unet/Visualizaciones/prueba/año1/visualizacion_test_2007.w51.png
Imagen guardada: /content/drive/MyDrive/Imagen_unet/Visualizaciones/prueba/año1/visualizacion_test_2007.

# Predicción

In [ ]:
import shutil

In [ ]:
torch.save(model.state_dict(), "unet_model_all.pth")
shutil.move("unet_model_all.pth", output_dir)

NameError: name 'shutil' is not defined